# etl_us_data.ipynb

Acquire a copy of the latest COVID-19 time series data and write that
data out into local CSV files in a format amenable to analysis with 
Pandas dataframes.

Input data sources:
* Primary data source: [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19).
* Secondary data source (for values missing from primary source): [New York Times "Coronavirus (Covid-19) Data in the United States" repository](https://github.com/nytimes/covid-19-data).
* Secondary data source (for values missing from primary source): [USAFacts Coronavirus Stats & Data](https://usafacts.org/issues/coronavirus/)

Output files produced:
* `outputs/us_counties.csv`: County-level time series data for the United States
* `outputs/us_counties_meta.json`: Column type metadata for reading `data/us_counties.csv` with `pd.read_csv()`

**Note:** You can redirect these output files by setting the environment variable `COVID_OUTPUTS_DIR` to a replacement for the prefix `outputs` in the above paths.

To read these files back in, use `pd.read_csv()`:
```python
with open("outputs/us_counties_meta.json") as f:
    cases_meta = json.load(f)
cases_meta["Date"] = "object"  # Workaround for pd.read_csv() not supporting parsing datetime64
cases_raw = pd.read_csv("../data/us_counties.csv", dtype=cases_meta, parse_dates=["Date"])
cases = cases_raw.set_index(["FIPS", "Date"], verify_integrity=True)
```


In [1]:
# Initialization boilerplate

# Import Python packages that this notebook uses.
import os
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
from datetime import datetime, date

# Local file of utility functions
import util

# Allow environment variables to override data file locations
_OUTPUTS_DIR = os.getenv("COVID_OUTPUTS_DIR", "outputs")
util.ensure_dir_exists(_OUTPUTS_DIR)  # create if necessary

# URLs for downloading the time series data directly from Github
_JH_BASE_URL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/" + \
               "csse_covid_19_data/csse_covid_19_time_series/"
_JH_CONFIRMED_URL = _JH_BASE_URL + "time_series_covid19_confirmed_US.csv"
_JH_DEATHS_URL = _JH_BASE_URL + "time_series_covid19_deaths_US.csv"

# Currently there are no data on recovered patients for the US.
# _JH_RECOVERED_URL = _JH_BASE_URL + "time_series_covid19_recovered_US.csv"

_NYT_BASE_URL = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/"
_NYT_CSV_URL = _NYT_BASE_URL + "us-counties.csv"

_USAFACTS_BASE_URL = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19"
_USAFACTS_CONFIRMED_URL = f"{_USAFACTS_BASE_URL}/covid_confirmed_usafacts.csv"
_USAFACTS_DEATHS_URL = f"{_USAFACTS_BASE_URL}/covid_deaths_usafacts.csv"

# First date present in the data set, and the format of these dates. 
# Hopefully this won't change as new data are added.
# NOTE: One file uses "01/22/20" and the other uses "1/22/20"
# so you need to filter with endswith() to find matches.
# Updated February 2021 to account for USAFacts changing their date format.
_FIRST_DATE_SUFFIXES = ["1/22/20", "2020-01-22"]
_DATE_FORMATS = ["%m/%d/%y", "%Y-%m-%d"]

# Johns Hopkins data

We pull the latest Johns Hopkins data from Github.

In [2]:
raw_confirmed = pd.read_csv(_JH_CONFIRMED_URL)
raw_deaths = pd.read_csv(_JH_DEATHS_URL)

# No "recovered" time series at the moment. Generate an empty
# time series the schema from the deaths
raw_recovered = raw_deaths.copy(deep=True)
for i in range(len(raw_recovered.columns)):
    if str(raw_recovered.columns[i]).endswith(_FIRST_DATE_SUFFIXES[0]):
        ts_start_index = i
        break
for c in raw_recovered.columns[ts_start_index:]:
    raw_recovered[c] = 0

raw_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15216,15288,15307,15325,15350,15366,15387,15409,15420,15431
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,54301,54391,54444,54481,54545,54614,54663,54700,54734,54763
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5402,5408,5409,5411,5413,5413,5422,5424,5426,5429
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,6294,6307,6314,6317,6331,6336,6345,6349,6351,6354
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,14516,14545,14556,14564,14575,14594,14632,14643,14663,14672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,9578,9578,9578,9603,9621,9642,9669,9692,9692,9692
3338,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,5569,5569,5569,5575,5584,5593,5598,5599,5599,5599
3339,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,2274,2274,2274,2284,2289,2297,2298,2303,2303,2303


In [3]:
# Some of the FIPS codes contain NA's:
raw_confirmed[raw_confirmed["FIPS"].isna()]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
1269,84070002,US,USA,840,NaN,Dukes and Nantucket,Massachusetts,US,41.406747,-70.687635,...,6680,6680,6680,6687,6690,6695,6701,6712,6712,6712
1306,84070005,US,USA,840,NaN,Federal Correctional Institution (FCI),Michigan,US,0.000000,0.000000,...,324,324,324,324,324,324,324,324,324,324
1338,84070004,US,USA,840,NaN,Michigan Department of Corrections (MDOC),Michigan,US,0.000000,0.000000,...,37302,37302,37302,37344,37344,37411,37411,37842,37842,37842
1593,84070003,US,USA,840,NaN,Kansas City,Missouri,US,39.099700,-94.578600,...,110110,110110,110110,110882,110833,110853,110916,110862,110862,110862
2956,84070015,US,USA,840,NaN,Bear River,Utah,US,41.521068,-113.083282,...,50593,50593,50593,50805,50839,50928,50972,51017,51017,51017
2961,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,...,19894,19894,19894,20051,20101,20189,20214,20286,20286,20286
2980,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,...,9807,9807,9807,9826,9836,9849,9864,9876,9876,9876
2981,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,62309,62309,62309,62504,62554,62631,62697,62785,62785,62785
2984,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,12461,12461,12461,12498,12507,12526,12542,12559,12559,12559
2992,84070020,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,...,72360,72360,72360,72638,72708,72804,72876,72970,72970,72970


## Filter the Johns Hopkins Data down to U.S. county-level statistics only.

Filter out the locations without county FIPS codes, since they don't
align properly with the county-level metadata this data set will be
joined with, and the number of cases involved is relatively small.

In [4]:
raw_confirmed = raw_confirmed[~raw_confirmed["FIPS"].isna()].copy()
raw_deaths = raw_deaths[~raw_deaths["FIPS"].isna()].copy()
raw_recovered = raw_recovered[~raw_recovered["FIPS"].isna()].copy()


# Probably due to the presence of NaNs, the remaining FIPS codes end
# up encoded as floating point numbers. Fix that.
for df in [raw_confirmed, raw_deaths, raw_recovered]:
    df["FIPS"] = df["FIPS"].astype("Int64")

raw_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,15216,15288,15307,15325,15350,15366,15387,15409,15420,15431
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,54301,54391,54444,54481,54545,54614,54663,54700,54734,54763
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,5402,5408,5409,5411,5413,5413,5422,5424,5426,5429
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,6294,6307,6314,6317,6331,6336,6345,6349,6351,6354
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,14516,14545,14556,14564,14575,14594,14632,14643,14663,14672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,84056039,US,USA,840,56039,Teton,Wyoming,US,43.935225,-110.589080,...,9578,9578,9578,9603,9621,9642,9669,9692,9692,9692
3338,84056041,US,USA,840,56041,Uinta,Wyoming,US,41.287818,-110.547578,...,5569,5569,5569,5575,5584,5593,5598,5599,5599,5599
3339,84090056,US,USA,840,90056,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,US,USA,840,56043,Washakie,Wyoming,US,43.904516,-107.680187,...,2274,2274,2274,2284,2289,2297,2298,2303,2303,2303


Remove locations that have FIPS codes but are not U.S. counties.

In [5]:
def counties_df(df):
    return df[(df["FIPS"] >= 1000)  # Territories have FIPS codes < 1000
              & (~df["Admin2"].isna())  # Countries don't have the "Admin2" field set
              & (df["Admin2"] != "Unassigned")  # States have Admin2 set to "Unassigned"
              & (df["FIPS"] <= 60000)  # Expatriates are coded by state in values > 80k
              ].copy()


county_confirmed = counties_df(raw_confirmed)
county_deaths = counties_df(raw_deaths)
county_recovered = counties_df(raw_recovered)

county_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,15216,15288,15307,15325,15350,15366,15387,15409,15420,15431
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,54301,54391,54444,54481,54545,54614,54663,54700,54734,54763
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,5402,5408,5409,5411,5413,5413,5422,5424,5426,5429
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,6294,6307,6314,6317,6331,6336,6345,6349,6351,6354
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,14516,14545,14556,14564,14575,14594,14632,14643,14663,14672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,84056037,US,USA,840,56037,Sweetwater,Wyoming,US,41.659439,-108.882788,...,10817,10817,10817,10867,10901,10920,10926,10937,10937,10937
3337,84056039,US,USA,840,56039,Teton,Wyoming,US,43.935225,-110.589080,...,9578,9578,9578,9603,9621,9642,9669,9692,9692,9692
3338,84056041,US,USA,840,56041,Uinta,Wyoming,US,41.287818,-110.547578,...,5569,5569,5569,5575,5584,5593,5598,5599,5599,5599
3340,84056043,US,USA,840,56043,Washakie,Wyoming,US,43.904516,-107.680187,...,2274,2274,2274,2284,2289,2297,2298,2303,2303,2303


## Rearrange the Johns Hopkins data into vertical time series

The time series in the raw data are spread across multiple columns. Rotate them by 90 degrees so that they are spread across rows.

In [6]:
def shred_time_series(df: pd.DataFrame, colname: str):
    """
    Turn a time series encoded as a range of columns into a time series
    encoded as a range of rows.

    This function hard-codes the column name mapping for Johns Hopkins 
    data, so it will only work on that data.

    :param df: Dataframe with a time series across the columns of each row,
     with an additional outer join indicator column at the very end.
    :param colname: Name of the new column where the time series should go

    :returns: A dataframe with one time series element per row.
     The returned dataframe will have a column called "Date" with the date
     of each time series element, and a column with the name `colname` with
     the associated value for each date.
    """
    ts_start_index = None
    date_format = None
    for i in range(len(df.columns)):
        for j in range(len(_DATE_FORMATS)):
            if str(df.columns[i]).endswith(_FIRST_DATE_SUFFIXES[j]):
                ts_start_index = i
                date_format = _DATE_FORMATS[j]
                break
        # break in python only exits one for loop scope
        if ts_start_index is not None:
            break
    if ts_start_index is None:
        raise ValueError(f"No timestamp columns found (schema is: {df.columns})")

    ts_matrix = df[df.columns[ts_start_index:]].to_numpy()
    ts_lists = ts_matrix.tolist()

    date_list = [datetime.strptime(s, date_format) for s in df.columns[ts_start_index:]]

    # Create a new dataframe where the time series is a list
    nested_df = df[df.columns[:ts_start_index]].copy()
    nested_df[colname] = ts_lists

    # Expand out the list and add the dates back.
    flat_df = nested_df.explode(colname)
    flat_df["Date"] = date_list * len(nested_df.index)
    return flat_df


shredded_confirmed = shred_time_series(county_confirmed, "Confirmed")
shredded_deaths = shred_time_series(county_deaths, "Deaths")
shredded_recovered = shred_time_series(county_recovered, "Recovered")
shredded_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Date
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-22
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-23
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-24
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-25
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1530,2022-02-16
3341,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1531,2022-02-17
3341,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1532,2022-02-18
3341,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1532,2022-02-19


## Merge the filtered and reformatted Johns Hopkins data into a single DataFrame

In [7]:
# Sort by FIPS code and Date and clean up some columns that don't match
# perfectly.
sorted_deaths = shredded_deaths.sort_values(["FIPS", "Date"])
sorted_confirmed = shredded_confirmed.sort_values(["FIPS", "Date"])
sorted_recovered = shredded_recovered.sort_values(["FIPS", "Date"])

# The "confirmed" time series is missing the "population" column that is
# present in the "deaths" and "recovered" time series.
# Add it back in.
sorted_confirmed["Population"] = sorted_deaths["Population"]

# The floating point numbers in the "Lat" and "Long_" fields also have
# some discrepancies due to rounding error. Use the values in the 
# "confirmed" time series as the gold standard.
sorted_deaths["Lat"] = sorted_confirmed["Lat"]
sorted_deaths["Long_"] = sorted_confirmed["Long_"]
sorted_recovered["Lat"] = sorted_confirmed["Lat"]
sorted_recovered["Long_"] = sorted_confirmed["Long_"]

# Now we can combine the three time series into a single table
combined = (
    sorted_confirmed
    .merge(sorted_deaths, how="outer")
    .merge(sorted_recovered, how="outer"))

# Check for missing data
missing_rows = combined[combined["Confirmed"].isna()]
if len(missing_rows.index) > 0:
    raise ValueError(f"Missing 'Confirmed' time series data for the following rows:\n{missing_rows}")

combined

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Confirmed,Date,Population,Deaths,Recovered
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-22,55869,0,0
1,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-23,55869,0,0
2,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-24,55869,0,0
3,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-25,55869,0,0
4,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",0,2020-01-26,55869,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392579,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1530,2022-02-16,6927,17,0
2392580,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1531,2022-02-17,6927,17,0
2392581,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1532,2022-02-18,6927,17,0
2392582,84056045,US,USA,840,56045,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1532,2022-02-19,6927,17,0


## Adjust data types and names of the columns in the merged DataFrame

In [8]:
# The outer joins in the previous cell convert some of the integer
# columns to object types. Fix that.

# Data types before:
combined.dtypes

UID                        int64
iso2                      object
iso3                      object
code3                      int64
FIPS                       Int64
Admin2                    object
Province_State            object
Country_Region            object
Lat                      float64
Long_                    float64
Combined_Key              object
Confirmed                 object
Date              datetime64[ns]
Population                 int64
Deaths                    object
Recovered                 object
dtype: object

In [9]:
combined["iso2"] = combined["iso2"].astype("string")
combined["iso3"] = combined["iso3"].astype("string")
combined["Admin2"] = combined["Admin2"].astype("string")
combined["Province_State"] = combined["Province_State"].astype("string")
combined["Country_Region"] = combined["Country_Region"].astype("string")
combined["Combined_Key"] = combined["Combined_Key"].astype("string")

combined["Confirmed"] = combined["Confirmed"].astype(np.int64)
combined["Deaths"] = combined["Deaths"].astype(np.int64)
combined["Recovered"] = combined["Recovered"].astype(np.int64)

# Data types after:
combined.dtypes

UID                        int64
iso2                      string
iso3                      string
code3                      int64
FIPS                       Int64
Admin2                    string
Province_State            string
Country_Region            string
Lat                      float64
Long_                    float64
Combined_Key              string
Confirmed                  int64
Date              datetime64[ns]
Population                 int64
Deaths                     int64
Recovered                  int64
dtype: object

In [10]:
# Massage the column names a bit and drop unnecessary columns
to_retain = combined[["Date", "FIPS", "Province_State", "Admin2", 
                      "Population",
                      "Confirmed", "Deaths", "Recovered"]]
renamed = to_retain.rename(columns={
    "Province_State": "State",
    "Admin2": "County"
})
renamed

,Date,FIPS,State,County,Population,Confirmed,Deaths,Recovered
0,2020-01-22,1001,Alabama,Autauga,55869,0,0,0
1,2020-01-23,1001,Alabama,Autauga,55869,0,0,0
2,2020-01-24,1001,Alabama,Autauga,55869,0,0,0
3,2020-01-25,1001,Alabama,Autauga,55869,0,0,0
4,2020-01-26,1001,Alabama,Autauga,55869,0,0,0
...,...,...,...,...,...,...,...,...
2392579,2022-02-16,56045,Wyoming,Weston,6927,1530,17,0
2392580,2022-02-17,56045,Wyoming,Weston,6927,1531,17,0
2392581,2022-02-18,56045,Wyoming,Weston,6927,1532,17,0
2392582,2022-02-19,56045,Wyoming,Weston,6927,1532,17,0


# New York Times data

Pull in additional data from the New York Times' data repository to use
for filling in holes in the primary JHU data set.

In [11]:
raw_nyt = pd.read_csv(_NYT_CSV_URL, parse_dates=["date"])
raw_nyt

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
2235984,2022-02-20,Sweetwater,Wyoming,56037.0,10937,122.0
2235985,2022-02-20,Teton,Wyoming,56039.0,9692,15.0
2235986,2022-02-20,Uinta,Wyoming,56041.0,5599,36.0
2235987,2022-02-20,Washakie,Wyoming,56043.0,2302,42.0


## Filter the New York Times data down to U.S. county-level statistics only.

In [12]:
# NY Times data also has some locations with FIPS codes of NaN:
raw_nyt[raw_nyt["fips"].isna()][["county", "state"]].drop_duplicates()

,county,state
416,New York City,New York
418,Unknown,Rhode Island
1511,Unknown,New Jersey
1858,Unknown,Puerto Rico
2267,Unknown,Virgin Islands
2422,Unknown,Guam
2929,Unknown,Maine
2950,Unknown,Massachusetts
4003,Unknown,Louisiana
4680,Unknown,Kentucky


In [13]:
# For now, drop the NaN FIPS codes like we did up above with the
# JHU data set.
raw_nyt = raw_nyt[~raw_nyt["fips"].isna()].copy()
raw_nyt["fips"] = raw_nyt["fips"].astype("int64")

# Also cast int-valued columns to nullable int.
raw_nyt["cases"] = raw_nyt["cases"].astype("Int64")
raw_nyt["deaths"] = raw_nyt["deaths"].astype("Int64")

# Rename the columns in preparation for joining with the primary
# data set.
nyt = raw_nyt.copy().rename(columns={
    "date": "Date",
    "county": "County",
    "state": "State",
    "fips": "FIPS",
    "cases": "Confirmed_NYT",
    "deaths": "Deaths_NYT"
})
nyt

,Date,County,State,FIPS,Confirmed_NYT,Deaths_NYT
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
2235984,2022-02-20,Sweetwater,Wyoming,56037,10937,122
2235985,2022-02-20,Teton,Wyoming,56039,9692,15
2235986,2022-02-20,Uinta,Wyoming,56041,5599,36
2235987,2022-02-20,Washakie,Wyoming,56043,2302,42


In [14]:
# Compare the number of distinct FIPS codes and dates in primary and
# secondary data sets.
print(f"Primary data set has {len(renamed['FIPS'].unique())} counties "
      f"and {len(renamed['Date'].unique())} dates.")
print(f"Secondary data set has {len(nyt['FIPS'].unique())} counties "
      f"and {len(nyt['Date'].unique())} dates.")

Primary data set has 3144 counties and 761 dates.
Secondary data set has 3220 counties and 762 dates.


# USAFacts data

We pull the latest data from the USAFacts CDN. 

The format of this data is very similar to that of the Johns Hopkins data, so the 
processing here is also similar.

In [15]:
raw_confirmed_usafacts = pd.read_csv(_USAFACTS_CONFIRMED_URL)
raw_deaths_usafacts = pd.read_csv(_USAFACTS_DEATHS_URL)

raw_confirmed_usafacts

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2021-08-07,2021-08-08,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15,2021-08-16
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,7694,7694,7788,7818,7854,7890,7931,7931,7931,8061
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,26822,26822,27477,27736,28020,28358,28629,28629,28629,29234
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,2585,2585,2629,2649,2681,2699,2732,2732,2732,2761
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,2973,2973,3016,3034,3056,3069,3095,3095,3095,3129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,5056,5056,5056,5090,5090,5108,5114,5127,5127,5127
3189,56039,Teton County,WY,56,0,0,0,0,0,0,...,3960,3960,3960,4003,4003,4014,4044,4068,4068,4068
3190,56041,Uinta County,WY,56,0,0,0,0,0,0,...,2498,2498,2498,2538,2538,2543,2559,2593,2593,2593
3191,56043,Washakie County,WY,56,0,0,0,0,0,0,...,950,950,950,958,958,960,968,967,967,967


## Filter the USAFacts Data down to U.S. county-level statistics only.

Filter out the locations without FIPS codes, since they don't
align properly with the county-level metadata this data set will be
joined with, and the number of cases involved is relatively small.

In [16]:
def counties_df(df):
    return df[(df["countyFIPS"] >= 1000)  # Territories and statewide remainders have FIPS codes < 1000
              ].copy()


county_confirmed_usafacts = counties_df(raw_confirmed_usafacts)
county_deaths_usafacts = counties_df(raw_deaths_usafacts)

county_confirmed_usafacts

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2021-08-07,2021-08-08,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15,2021-08-16
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,7694,7694,7788,7818,7854,7890,7931,7931,7931,8061
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,26822,26822,27477,27736,28020,28358,28629,28629,28629,29234
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,2585,2585,2629,2649,2681,2699,2732,2732,2732,2761
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,2973,2973,3016,3034,3056,3069,3095,3095,3095,3129
5,1009,Blount County,AL,1,0,0,0,0,0,0,...,7542,7542,7603,7631,7658,7696,7758,7758,7758,7833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,5056,5056,5056,5090,5090,5108,5114,5127,5127,5127
3189,56039,Teton County,WY,56,0,0,0,0,0,0,...,3960,3960,3960,4003,4003,4014,4044,4068,4068,4068
3190,56041,Uinta County,WY,56,0,0,0,0,0,0,...,2498,2498,2498,2538,2538,2543,2559,2593,2593,2593
3191,56043,Washakie County,WY,56,0,0,0,0,0,0,...,950,950,950,958,958,960,968,967,967,967


## Rearrange the USAFacts data into vertical time series

The time series in the raw data are spread across multiple columns. Rotate them by 90 degrees so that they are spread across rows.

In [17]:
shredded_confirmed_usafacts = shred_time_series(county_confirmed_usafacts, "Confirmed")
shredded_deaths_usafacts = shred_time_series(county_deaths_usafacts, "Deaths")
shredded_confirmed_usafacts

,countyFIPS,County Name,State,StateFIPS,Confirmed,Date
1,1001,Autauga County,AL,1,0,2020-01-22
1,1001,Autauga County,AL,1,0,2020-01-23
1,1001,Autauga County,AL,1,0,2020-01-24
1,1001,Autauga County,AL,1,0,2020-01-25
1,1001,Autauga County,AL,1,0,2020-01-26
...,...,...,...,...,...,...
3192,56045,Weston County,WY,56,700,2021-08-12
3192,56045,Weston County,WY,56,700,2021-08-13
3192,56045,Weston County,WY,56,705,2021-08-14
3192,56045,Weston County,WY,56,705,2021-08-15


In [18]:
shredded_deaths_usafacts[shredded_deaths_usafacts["countyFIPS"] == 8014]

,countyFIPS,County Name,State,StateFIPS,Deaths,Date
257,8014,City and County of Broomfield,CO,8,0,2020-01-22
257,8014,City and County of Broomfield,CO,8,0,2020-01-23
257,8014,City and County of Broomfield,CO,8,0,2020-01-24
257,8014,City and County of Broomfield,CO,8,0,2020-01-25
257,8014,City and County of Broomfield,CO,8,0,2020-01-26
...,...,...,...,...,...,...
257,8014,City and County of Broomfield,CO,8,77,2021-08-12
257,8014,City and County of Broomfield,CO,8,77,2021-08-13
257,8014,City and County of Broomfield,CO,8,77,2021-08-14
257,8014,City and County of Broomfield,CO,8,77,2021-08-15


In [19]:
shredded_confirmed_usafacts[shredded_confirmed_usafacts["countyFIPS"] == 8014]

,countyFIPS,County Name,State,StateFIPS,Confirmed,Date
257,8014,City and County of Broomfield,CO,8,0,2020-01-22
257,8014,City and County of Broomfield,CO,8,0,2020-01-23
257,8014,City and County of Broomfield,CO,8,0,2020-01-24
257,8014,City and County of Broomfield,CO,8,0,2020-01-25
257,8014,City and County of Broomfield,CO,8,0,2020-01-26
...,...,...,...,...,...,...
257,8014,City and County of Broomfield,CO,8,5340,2021-08-12
257,8014,City and County of Broomfield,CO,8,5360,2021-08-13
257,8014,City and County of Broomfield,CO,8,5360,2021-08-14
257,8014,City and County of Broomfield,CO,8,5360,2021-08-15


## Merge the filtered and reformatted USAFacts data into a single DataFrame

In [20]:
# Sort by FIPS code and Date prior to joining
sorted_deaths_usafacts = shredded_deaths_usafacts.sort_values(["countyFIPS", "Date"])
sorted_confirmed_usafacts = shredded_confirmed_usafacts.sort_values(["countyFIPS", "Date"])

# Now we can combine the time series into a single table
combined_usafacts = sorted_confirmed_usafacts.merge(
    sorted_deaths_usafacts[["countyFIPS", "Date", "Deaths"]], 
    how="outer", on=["countyFIPS", "Date"])

combined_usafacts

,countyFIPS,County Name,State,StateFIPS,Confirmed,Date,Deaths
0,1001,Autauga County,AL,1,0,2020-01-22,0
1,1001,Autauga County,AL,1,0,2020-01-23,0
2,1001,Autauga County,AL,1,0,2020-01-24,0
3,1001,Autauga County,AL,1,0,2020-01-25,0
4,1001,Autauga County,AL,1,0,2020-01-26,0
...,...,...,...,...,...,...,...
1800361,56045,Weston County,WY,56,700,2021-08-12,6
1800362,56045,Weston County,WY,56,700,2021-08-13,6
1800363,56045,Weston County,WY,56,705,2021-08-14,6
1800364,56045,Weston County,WY,56,705,2021-08-15,6


## Adjust data types and names of the columns in the merged DataFrame

In [21]:
# Data types before:
combined_usafacts.dtypes

countyFIPS              int64
County Name            object
State                  object
StateFIPS               int64
Confirmed              object
Date           datetime64[ns]
Deaths                 object
dtype: object

In [22]:
# Encode strings with the Pandas string type
combined_usafacts["County Name"] = combined_usafacts["County Name"].astype("string")
combined_usafacts["State"] = combined_usafacts["State"].astype("string")

# Encode integer fields containing NaNs using the Pandas nullable int type
combined_usafacts["Confirmed"] = combined_usafacts["Confirmed"].astype("Int64")
combined_usafacts["Deaths"] = combined_usafacts["Deaths"].astype("Int64")

# Data types after:
combined_usafacts.dtypes

countyFIPS              int64
County Name            string
State                  string
StateFIPS               int64
Confirmed               Int64
Date           datetime64[ns]
Deaths                  Int64
dtype: object

In [23]:
# Massage the column names a bit and drop unnecessary columns
to_retain_usafacts = combined_usafacts[
    ["Date", "countyFIPS", "County Name", 
     "State", "Confirmed", "Deaths"]]
renamed_usafacts = to_retain_usafacts.rename(columns={
    "countyFIPS": "FIPS",
    "County Name": "County",
    "Confirmed": "Confirmed_USAFacts",
    "Deaths": "Deaths_USAFacts",
})
renamed_usafacts

,Date,FIPS,County,State,Confirmed_USAFacts,Deaths_USAFacts
0,2020-01-22,1001,Autauga County,AL,0,0
1,2020-01-23,1001,Autauga County,AL,0,0
2,2020-01-24,1001,Autauga County,AL,0,0
3,2020-01-25,1001,Autauga County,AL,0,0
4,2020-01-26,1001,Autauga County,AL,0,0
...,...,...,...,...,...,...
1800361,2021-08-12,56045,Weston County,WY,700,6
1800362,2021-08-13,56045,Weston County,WY,700,6
1800363,2021-08-14,56045,Weston County,WY,705,6
1800364,2021-08-15,56045,Weston County,WY,705,6


# Merge the three data sets into a single DataFrame

In [24]:
# Outer-join the three data sets
to_write = (
    renamed
    # Don't use the county names, because they differ across data sets
    .merge(nyt[["Date", "FIPS", "Confirmed_NYT", "Deaths_NYT"]], how="left")
    .merge(renamed_usafacts[["Date", "FIPS", "Confirmed_USAFacts", "Deaths_USAFacts"]], 
           on=["Date", "FIPS"], how="left")
)
to_write

,Date,FIPS,State,County,Population,Confirmed,Deaths,Recovered,Confirmed_NYT,Deaths_NYT,Confirmed_USAFacts,Deaths_USAFacts
0,2020-01-22,1001,Alabama,Autauga,55869,0,0,0,<NA>,<NA>,0,0
1,2020-01-23,1001,Alabama,Autauga,55869,0,0,0,<NA>,<NA>,0,0
2,2020-01-24,1001,Alabama,Autauga,55869,0,0,0,<NA>,<NA>,0,0
3,2020-01-25,1001,Alabama,Autauga,55869,0,0,0,<NA>,<NA>,0,0
4,2020-01-26,1001,Alabama,Autauga,55869,0,0,0,<NA>,<NA>,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2392579,2022-02-16,56045,Wyoming,Weston,6927,1530,17,0,1530,17,<NA>,<NA>
2392580,2022-02-17,56045,Wyoming,Weston,6927,1531,17,0,1531,17,<NA>,<NA>
2392581,2022-02-18,56045,Wyoming,Weston,6927,1532,17,0,1532,17,<NA>,<NA>
2392582,2022-02-19,56045,Wyoming,Weston,6927,1532,17,0,1532,17,<NA>,<NA>


# Write out the merged data to a CSV file

In [25]:
# Write the data out to a CSV file + a JSON file of type info.
output_csv_data_file = os.path.join(_OUTPUTS_DIR, "us_counties.csv")
print(f"Writing data to {output_csv_data_file}")
to_write.to_csv(output_csv_data_file, index=False)
col_type_mapping = {
    key: str(value) for key, value in to_write.dtypes.iteritems()
}
output_json_data_file = os.path.join(_OUTPUTS_DIR, "us_counties_meta.json")
print(f"Writing metadata to {output_json_data_file}")
with open(output_json_data_file, "w") as f:
    json.dump(col_type_mapping, f)

Writing data to outputs/us_counties.csv


Writing metadata to outputs/us_counties_meta.json
